# mastodon API followers

read data of accounts you are following and store it in excel file.

+ utilizes `Mastodon.py` package to read account information
+ `pandas` is used to create excel file
  + sheet for accounts following
  + sheet for status per day, find loud and mute accounts
+ read follower data
+ calculate metrics
  + total followers
  + lost followers
  + gained followers
  + changes on followers status, follows and followers
+ write/update excel file overview and total followers
+ write follower changes excel file


background links, references

+ [Mastodon.py](https://pypi.org/project/Mastodon.py/)
  + [pypi](https://pypi.org/project/Mastodon.py/)
  + [doc](https://mastodonpy.readthedocs.io/en/stable/)
  + [repo](https://github.com/halcy/Mastodon.py)
+ [mastodon](https://joinmastodon.org/)
  + [API getting started](https://docs.joinmastodon.org/client/intro/)
  + [pagination](https://docs.joinmastodon.org/api/guidelines/#pagination)

**setup**

```shell
pip install -r requirements.txt
```

**THE MIT license (MIT)**

Copyright © 2022 Walter Obweger (WRO)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
sSecretFN='.env.py.secret'
sOutFN='me.xlsx'
sShtMtr='metric'
sShtFlr='follower'

In [ ]:
lColMtr=['zStamp','iStatus','iFollows','iFollower']
lColMtrFmt=[
    ('A:A',20),
    ('B:D',8),

]

In [ ]:
lColFlr=['id','acct','username','iStatus','iFollows','iFollower']
lColFlrFmt=[
    ('A:A',30),
    ('B:B',30),
    ('C:C',40),
    ('D:F',8),

]

In [ ]:
lColFlrSame=['id','acct','username','iStatus','iStatusPrv','iStatusDt','iFollows','iFollowsPrv','iFollowsDt','iFollower','iFollowerPrv','iFollowerDt',]
lColFlrSameFmt=[
    ('A:A',30),
    ('B:B',30),
    ('C:C',40),
    ('D:F',8),
    ('G:I',8),
    ('J:L',8),

]

In [ ]:
import os
import time
import datetime
import pprint
import traceback
import random
import pandas

from dotenv import load_dotenv
from mastodon import Mastodon

In [ ]:
dfMtr=None
dfFlr=None
if os.path.exists(sOutFN):
    oWrk=pandas.ExcelFile(sOutFN)
    if sShtMtr in oWrk.sheet_names:
        dfMtr=oWrk.parse(sShtMtr)
    if sShtFlr in oWrk.sheet_names:
        dfFlr=oWrk.parse(sShtFlr)
if dfMtr is None:
    dfMtr=pandas.DataFrame(columns=lColMtr)
if dfFlr is None:
    dfFlr=pandas.DataFrame(columns=lColFlr)

In [ ]:
iVerbose=20

In [ ]:
zNow=datetime.datetime.now()

In [ ]:
load_dotenv(sSecretFN)
print(os.environ.get('api_base_url'))

In [ ]:
oMstdn = Mastodon(
    access_token = os.environ.get('access_token'),
    api_base_url = os.environ.get('api_base_url')
)

In [ ]:
sVer=oMstdn.retrieve_mastodon_version()
print(sVer)

In [ ]:
dMe=oMstdn.me()

In [ ]:
print(list(dMe.keys()))

add processing here

In [ ]:
dMtr={
    'zStamp':zNow.strftime('%Y-%m-%d %H:%M'),
    'iStatus':dMe['statuses_count'],
    'iFollower':dMe['followers_count'],
    'iFollows':dMe['following_count'],

}
dfMtrTmp=pandas.DataFrame(data=[dMtr])

In [ ]:
dfMtr=pandas.concat([dfMtr,dfMtrTmp],ignore_index=True)

In [ ]:
dfMtr.tail()

**follower**

iterate over all followers

+ build total ever followers
+ calc same followers and their changes
+ calc lost followers
+ calc gained followers

In [ ]:
lFlr=[]

In [ ]:
def addFollower(lFollower,lFlr):
    for dFollower in lFollower:
        dFlr={
            'id':dFollower.id,
            'acct':dFollower.acct,
            'username':dFollower.username,
            'iStatus':dFollower.statuses_count,
            'iFollower':dFollower.followers_count,
            'iFollows':dFollower.following_count,
        }
        lFlr.append(dFlr)

In [ ]:
lFollower=oMstdn.account_followers(dMe.id)

In [ ]:
try:
    while (lFollower is not None):
        addFollower(lFollower,lFlr)
        lFollowerNext=oMstdn.fetch_next(lFollower)
        if iVerbose>20:
            print(len(lFollowerNext))
        lFollower=lFollowerNext
        time.sleep(0.1+random.random()*0.25)
except:
    print(traceback.format_exc())
if iVerbose>0:
    print('follower count:%d'%(len(lFlr)))

In [ ]:
dfFlrTmp=pandas.DataFrame(data=lFlr)

In [ ]:
print(dfFlrTmp.shape)
dfFlrTmp.head()

In [ ]:
dfFlrTmp['id']=dfFlrTmp['id'].astype(str)
dfFlr['id']=dfFlr['id'].astype(str)

In [ ]:
dfFlr.shape

In [ ]:
print(dfFlr.shape)
dfFlr.head()

In [ ]:
dfFlrTmpIdx=dfFlrTmp.set_index(['acct'])
dfFlrIdx=dfFlr.set_index(['acct'])

In [ ]:
dfFlrSame=dfFlrTmpIdx.join(dfFlrIdx,how='inner',rsuffix="Prv")

In [ ]:
print(dfFlrSame.shape)
dfFlrSame.head()

In [ ]:
def calcDeltaTime(x):
    # calculate delta time status
    iPrv=x['iStatusPrv']
    iNxt=x['iStatus']
    iDelta=iNxt-iPrv
    x['iStatusDt']=iDelta
    # calculate delta time follower
    iPrv=x['iFollowerPrv']
    iNxt=x['iFollower']
    iDelta=iNxt-iPrv
    x['iFollowerDt']=iDelta
    # calculate delta time follows
    iPrv=x['iFollowsPrv']
    iNxt=x['iFollows']
    iDelta=iNxt-iPrv
    x['iFollowsDt']=iDelta
    return x

In [ ]:
dfFlrSame=dfFlrSame.apply(calcDeltaTime,axis=1)

In [ ]:
dfFlrSame.head()

In [ ]:
dfFlrSame=dfFlrSame.drop(['idPrv','usernamePrv'],axis=1)

In [ ]:
dfFlrNew=dfFlrTmpIdx[~dfFlrTmpIdx.isin(dfFlrSame).all(1)]
print(dfFlrNew.shape)
dfFlrNew.head()

In [ ]:
dfFlrDel=dfFlrIdx[~dfFlrIdx.isin(dfFlrTmpIdx).all(1)]
print(dfFlrDel.shape)
dfFlrDel.head()

In [ ]:
print(dfFlrIdx.shape)
print(dfFlrTmpIdx.shape)

In [ ]:
dfFlrIdx.head()

In [ ]:
dfFlrTmpIdx.head()

calc total followers

In [ ]:
dfFlrAll=pandas.concat([dfFlrTmp,dfFlr,],ignore_index=True,join='outer',keys=['acct'])
print(dfFlrAll.shape)

In [ ]:
dfFlrAll['id']=dfFlrAll['id'].astype(str)

In [ ]:
dfFlrAll.sort_values(['acct'],inplace=True)

In [ ]:
dfFlrAll.head()

output helpers

In [ ]:
def applyFmt(oExcelWr,sSht,lCol):
    oBok=oExcelWr.book
    oSht=oExcelWr.sheets[sSht]
    oFmt=oBok.add_format({
                        'text_wrap':False,
                        'align':'left',
                        'valign':'top',
                        })
    for sAdr,iWidth in lCol:
        oSht.set_column(sAdr,iWidth,oFmt)

sort data for output

In [ ]:
dfMtr.sort_values(['zStamp'],ascending=False,inplace=True)

write following information

In [ ]:
sShtFlrCur="flr.cur"
sShtFlrSame="flr.same"
sShtFlrDel="flr.lost"
sShtFlrNew="flr.gained"

In [ ]:
dfFlrNew=dfFlrNew.reset_index()
dfFlrNew=dfFlrNew.reindex(columns=lColFlr)
dfFlrDel=dfFlrDel.reset_index()
dfFlrDel=dfFlrDel.reindex(columns=lColFlr)
dfFlrSame=dfFlrSame.reset_index()
dfFlrSame=dfFlrSame.reindex(columns=lColFlrSame)


In [ ]:
dfFlrNew['id']=dfFlrNew['id'].astype(str)
dfFlrDel['id']=dfFlrDel['id'].astype(str)
dfFlrTmp['id']=dfFlrTmp['id'].astype(str)

In [ ]:
sNow=zNow.strftime('%Y%m%d_%H%M')
sOutNowFN="me.%s.xlsx"%(sNow)

In [ ]:
#sOutToDayFN="me.tmp.xlsx"
with pandas.ExcelWriter(sOutFN,mode='w') as oXlsWr:
    dfMtr.to_excel(oXlsWr,sheet_name=sShtMtr,index=False)
    applyFmt(oXlsWr,sShtMtr,lColMtrFmt)

    dfFlrAll.to_excel(oXlsWr,sheet_name=sShtFlr,index=False)
    applyFmt(oXlsWr,sShtFlr,lColFlrFmt)

with pandas.ExcelWriter(sOutNowFN,mode='w') as oXlsWr:

    dfFlrTmp.to_excel(oXlsWr,sheet_name=sShtFlrCur,index=False)
    applyFmt(oXlsWr,sShtFlrCur,lColFlrFmt)

    dfFlrSame.to_excel(oXlsWr,sheet_name=sShtFlrSame,index=False)
    applyFmt(oXlsWr,sShtFlrSame,lColFlrSameFmt)

    dfFlrDel.to_excel(oXlsWr,sheet_name=sShtFlrDel,index=False)
    applyFmt(oXlsWr,sShtFlrDel,lColFlrFmt)

    dfFlrNew.to_excel(oXlsWr,sheet_name=sShtFlrNew,index=False)
    applyFmt(oXlsWr,sShtFlrNew,lColFlrFmt)
